# Setup

In [ ]:
import asyncio
import queue


In [ ]:
import memory_profiler
import time

def time_mem_decorator(func):                                                                                            
    def out(*args, **kwargs):                                                                                            
        m1 = memory_profiler.memory_usage()
        t1 = time.time()
        
        result = func(*args, **kwargs)
        
        t2 = time.time()
        m2 = memory_profiler.memory_usage()
        time_diff = t2 - t1
        mem_diff = m2[0] - m1[0]
        print(f"It took {time_diff} Secs and {mem_diff} Mb to execute this function.")
        return(result)
    return out 

# Executing multiple tasks

In [ ]:
def calculation(x):
    print(f"Running calculation with x = {x}.")
    time.sleep(x)
    print(f"Task with x = {x} is done.")

In [ ]:
@time_mem_decorator
def synchronous_work():
    task_queue = queue.Queue()
    
    for t in [2, 5, 10, 4, 6]:
        task_queue.put(t)
        
    while not task_queue.empty():
        x = task_queue.get()
        calculation(x)
    


In [ ]:
synchronous_work()

If you have multiple workers, tasks don't have to wait for each other.

In [ ]:
async def async_calculation(x):
    print(f"Running calculation with x = {x}.")
    await asyncio.sleep(x)
    print(f"Task with x = {x} is done.")

In [ ]:
async def worker(task_queue):
    while not task_queue.empty():
        x = await task_queue.get()
        await async_calculation(x)
        task_queue.task_done()

In [ ]:
async def asynchronous_work():
    t1 = time.time()
    task_queue = asyncio.Queue()
    
    for t in [2, 5, 10, 4, 6]:
        await task_queue.put(t)
       
    await asyncio.gather(
        asyncio.create_task(worker(task_queue)),
        asyncio.create_task(worker(task_queue))
    )
    t2 = time.time()
    
    print(f"It took {t2 - t1} Secs to execute this function.")

In [ ]:
event_loop = asyncio.get_event_loop()
await event_loop.create_task(asynchronous_work())